# Импорт библиотек

In [55]:
import re
import nltk
import string
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df = pd.read_csv('News.csv')
df.head(10)

,headline,category,authors,short_description
0,There Were 2 Mass Shootings In Texas Last Wee...,CRIME,Melissa Jeltsen,She left her husband. He killed their childre...
1,Will Smith Joins Diplo And Nicky Jam For The ...,ENTERTAINMENT,Andy McDonald,Of course it has a song.
2,Hugh Grant Marries For The First Time At Age ...,ENTERTAINMENT,Ron Dicker,The actor and his longtime girlfriend Anna Eb...
3,Jim Carrey Blasts 'Castrato' Adam Schiff And ...,ENTERTAINMENT,Ron Dicker,The actor gives Dems an ass-kicking for not f...
4,Julianna Margulies Uses Donald Trump Poop Bag...,ENTERTAINMENT,Ron Dicker,The Dietland actress said using the bags is a...
5,Morgan Freeman 'Devastated' That Sexual Haras...,ENTERTAINMENT,Ron Dicker,It is not right to equate horrific incidents ...
6,Donald Trump Is Lovin' New McDonald's Jingle ...,ENTERTAINMENT,Ron Dicker,It's catchy all right.
7,What To Watch On Amazon Prime That\u2019s New...,ENTERTAINMENT,Todd Van Luling,There's a great mini-series joining this week.
8,Mike Myers Reveals He'd 'Like To' Do A Fourth...,ENTERTAINMENT,Andy McDonald,Myer's kids may be pushing for a new Powers f...
9,What To Watch On Hulu That\u2019s New This We...,ENTERTAINMENT,Todd Van Luling,You're getting a recent Academy Award-winning...


In [3]:
df.dropna() # удаляем пустые значении
df

,headline,category,authors,short_description
0,There Were 2 Mass Shootings In Texas Last Wee...,CRIME,Melissa Jeltsen,She left her husband. He killed their childre...
1,Will Smith Joins Diplo And Nicky Jam For The ...,ENTERTAINMENT,Andy McDonald,Of course it has a song.
2,Hugh Grant Marries For The First Time At Age ...,ENTERTAINMENT,Ron Dicker,The actor and his longtime girlfriend Anna Eb...
3,Jim Carrey Blasts 'Castrato' Adam Schiff And ...,ENTERTAINMENT,Ron Dicker,The actor gives Dems an ass-kicking for not f...
4,Julianna Margulies Uses Donald Trump Poop Bag...,ENTERTAINMENT,Ron Dicker,The Dietland actress said using the bags is a...
...,...,...,...,...
200848,RIM CEO Thorsten Heins' 'Significant' Plans F...,TECH,Reuters Reuters,Verizon Wireless and AT&T are already promoti...
200849,Maria Sharapova Stunned By Victoria Azarenka ...,SPORTS,,Afterward Azarenka more effusive with the pre...
200850,Giants Over Patriots Jets Over Colts Among M...,SPORTS,,Leading up to Super Bowl XLVI the most talked...
200851,Aldon Smith Arrested,SPORTS,49ers Linebacker Busted For DUI authors,//www.huffingtonpost.com/entry/aldon-smith-arr...


In [5]:
df["category"].unique()

array([' CRIME  ', ' ENTERTAINMENT  ', ' WORLD NEWS  ', ' IMPACT  ',
       ' POLITICS  ', ' WEIRD NEWS  ', ' BLACK VOICES  ', ' WOMEN  ',
       ' COMEDY  ', ' QUEER VOICES  ', ' SPORTS  ', ' BUSINESS  ',
       ' TRAVEL  ', ' MEDIA  ', ' TECH  ', ' RELIGION  ', ' SCIENCE  ',
       ' LATINO VOICES  ', ' EDUCATION  ', ' COLLEGE  ', ' PARENTS  ',
       ' ARTS & CULTURE  ', ' STYLE  ', ' GREEN  ', ' TASTE  ',
       ' HEALTHY LIVING  ', ' THE WORLDPOST  ', ' GOOD NEWS  ',
       ' WORLDPOST  ', ' FIFTY  ', ' ARTS  ', ' WELLNESS  ',
       ' PARENTING  ', ' HOME & LIVING  ', ' STYLE & BEAUTY  ',
       ' DIVORCE  ', ' WEDDINGS  ', ' FOOD & DRINK  ', ' MONEY  ',
       ' ENVIRONMENT  ', ' CULTURE & ARTS  '], dtype=object)

In [9]:
data_which = [' CRIME  ', ' WORLD NEWS  ', ' POLITICS  ', ' SPORTS  ',' TECH  ', ' RELIGION  ', ' SCIENCE  ',
              ' EDUCATION  ',' ARTS  ', ' FOOD & DRINK  ', ' MONEY  ']

data_drops = []
for val in df["category"]: # оставляем нужные данные, как многие данные нам не нужны(лично для меня)
    if val not in data_which:
        data_drops.append(val)

In [15]:
df2 = df[~df['category'].isin(data_drops)]

data_replace = {cat: i for i, cat in enumerate(df2["category"].unique())}

df2["new_category"] = df2["category"].replace(data_replace) # переобразую строку в цифровые данные
df2.head()

<ipython-input-15-905dded41d7c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["new_category"] = df2["category"].replace(data_replace)


,headline,category,authors,short_description,new_category
0,There Were 2 Mass Shootings In Texas Last Wee...,CRIME,Melissa Jeltsen,She left her husband. He killed their childre...,0
11,South Korean President Meets North Korea's Ki...,WORLD NEWS,,The two met to pave the way for a summit betw...,1
13,Trump's Crackdown On Immigrant Parents Puts M...,POLITICS,Elise Foley and Roque Planas,Last month a Health and Human Services offici...,2
14,'Trump's Son Should Be Concerned',POLITICS,FBI Obtained Wiretaps Of Putin Ally Who Met W...,//www.huffingtonpost.com/entry/fbi-wiretaps-pu...,2
15,Edward Snowden,POLITICS,There's No One Trump Loves More Than Vladimir...,//www.huffingtonpost.com/entry/edward-snowden-...,2


In [27]:
database = df2.loc[:, ['headline', "new_category"]]

for vals in df2["category"].unique():
    print("{} count {}".format(vals, len(df2[df2["category"]==vals]))) # смотрям сколько данных найдены по значении

 CRIME   count 3405
 WORLD NEWS   count 2177
 POLITICS   count 32739
 SPORTS   count 4884
 TECH   count 2082
 RELIGION   count 2556
 SCIENCE   count 2178
 EDUCATION   count 1004
 ARTS   count 1509
 FOOD & DRINK   count 6226
 MONEY   count 1707


In [30]:
min_value = 1000

category_counts = database['new_category'].value_counts()

new_data = pd.DataFrame(columns=database.columns)

for category in category_counts.index:
    category_data = database[database['new_category'] == category] # убираем чтобы остались данные только по категориям 1000
    if len(category_data) >= min_value: # по всем данные останеться только 1000 примеров
        new_data = new_data.append(category_data.sample(min_value)) # 1000 потому что у меня самый минимум был 1004
    else:
        new_data = new_data.append(category_data)
new_data

<ipython-input-30-2c5ab6336220>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data = new_data.append(category_data.sample(min_value))
<ipython-input-30-2c5ab6336220>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data = new_data.append(category_data.sample(min_value))
<ipython-input-30-2c5ab6336220>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data = new_data.append(category_data.sample(min_value))
<ipython-input-30-2c5ab6336220>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data = new_data.append(category_data.sample(min_value))
<ipython-input-30-2c5ab6336220>:10: FutureWarning: The frame.append method i

,headline,new_category
94769,Women's Reproductive Liberty,2
53488,Marco Rubio Questions Ruth Bader Ginsburg's C...,2
70027,Jeb Bush Rules Out Third Party Run Even If Do...,2
39984,Hate Speech vs. Free Speech,2
62128,Ted Cruz Defends \u2018New York Values\u2019 ...,2
...,...,...
120511,The Power of 'Yes' authors,7
62469,The Vitality of Collegiate Recovery authors,7
91949,A Gay 6-Year-Old's Thank You To His First Gra...,7
53282,Nation's Largest Teachers Union Endorses Teac...,7


# Текстовые данны токенизация, стоп слова и т.д

In [35]:
st = nltk.PorterStemmer()
lm = nltk.WordNetLemmatizer()

stop_words = set(stopwords.words("english")) # стоп слова в англ

def clean(text):
    n_zhok = text.replace("\n", "")
    text_tazar = "".join([i for i in n_zhok if i not in string.punctuation]) # убираем не нужные символы
    token = re.split("\W+", text)
    stop_w = [j.lower() for j in token if j not in stop_words] # стоп слова
    sting = [st.stem(w) for w in stop_w] # стеммизация
    lmize = " ".join([lm.lemmatize(k) for k in sting]) # лиммитизация
    return lmize

new_data["new_text"] = new_data["headline"].apply(lambda x: clean(x)) # делаем новый столбец
data = new_data.loc[:, ["new_category", "new_text"]]
data

,new_category,new_text
94769,2,woman reproduct liberti
53488,2,marco rubio question ruth bader ginsburg crit...
70027,2,jeb bush rule out third parti run even if don...
39984,2,hate speech v free speech
62128,2,ted cruz defend u2018new york valu u2019 jab ...
...,...,...
120511,7,the power ye author
62469,7,the vital collegi recoveri author
91949,7,a gay 6 year old thank you to hi first grade ...
53282,7,nation largest teacher union endors teach cli...


In [41]:
data.new_text

94769                               woman reproduct liberti
53488      marco rubio question ruth bader ginsburg crit...
70027      jeb bush rule out third parti run even if don...
39984                             hate speech v free speech
62128      ted cruz defend u2018new york valu u2019 jab ...
                                ...                        
120511                                  the power ye author
62469                     the vital collegi recoveri author
91949      a gay 6 year old thank you to hi first grade ...
53282      nation largest teacher union endors teach cli...
122609     a tale two kindergarten well three now that i...
Name: new_text, Length: 11000, dtype: object

In [44]:
import collections
from collections import Counter

def count_w(text):
    count = Counter()
    for t in text:
        for word in t.split():
            count[word] += 1
    return count

counter = count_w(data.new_text)

In [48]:
kk = len(counter)
kk

10699

In [61]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data["new_text"], data["new_category"], train_size=0.77)

In [65]:
y_test

82302     5
3437      2
117565    5
137951    6
150344    0
         ..
177592    9
13640     1
60154     6
33365     2
172304    3
Name: new_category, Length: 2530, dtype: object

In [50]:
token = Tokenizer(num_words=len(counter))
token.fit_on_texts(X_train) # делаем токен давая словам индекс

In [52]:
wordss = token.word_index
wordss

{'author': 1,
 'the': 2,
 'to': 3,
 'in': 4,
 'a': 5,
 'of': 6,
 'for': 7,
 'is': 8,
 'on': 9,
 'and': 10,
 'with': 11,
 'you': 12,
 'new': 13,
 'trump': 14,
 'it': 15,
 'how': 16,
 'from': 17,
 'after': 18,
 'at': 19,
 'your': 20,
 'are': 21,
 'be': 22,
 'thi': 23,
 'say': 24,
 'about': 25,
 'what': 26,
 'that': 27,
 'video': 28,
 'year': 29,
 'school': 30,
 'u': 31,
 'whi': 32,
 'can': 33,
 'by': 34,
 'photo': 35,
 'make': 36,
 'woman': 37,
 'more': 38,
 'will': 39,
 'u2019': 40,
 'we': 41,
 'world': 42,
 'get': 43,
 'man': 44,
 'show': 45,
 'day': 46,
 'over': 47,
 'who': 48,
 'kill': 49,
 'not': 50,
 's': 51,
 'recip': 52,
 'student': 53,
 'educ': 54,
 'first': 55,
 'may': 56,
 'american': 57,
 'polic': 58,
 'way': 59,
 'up': 60,
 'like': 61,
 'have': 62,
 '5': 63,
 'out': 64,
 'could': 65,
 'best': 66,
 'take': 67,
 'hi': 68,
 'one': 69,
 'shoot': 70,
 'all': 71,
 'food': 72,
 'teacher': 73,
 'want': 74,
 'an': 75,
 'here': 76,
 'watch': 77,
 'time': 78,
 'pope': 79,
 'just': 80,


In [66]:
train_seq = token.texts_to_sequences(X_train) # индексация трен. данных
val_seq = token.texts_to_sequences(X_test) # индексация тест. данных

In [54]:
print(train_seq[1:10])

[[2381, 184, 222, 172, 305, 4, 2064, 340, 798, 1], [1807, 3568, 4943, 4944, 68, 640, 27, 1171, 2858, 326, 2859, 1], [230], [3569, 1299, 572, 269, 197, 44, 11, 1299, 2065, 1], [4945, 151, 13, 665, 306, 1300, 1], [16, 451, 22, 372, 4946, 1], [1808, 8, 36, 4947, 11, 5, 1172, 4, 924, 1], [173, 49, 862, 2860, 1173, 11, 474, 925, 61, 1], [4948, 4949, 8, 318, 5, 863, 135, 69, 74, 1]]


In [139]:
train_ped = pad_sequences(train_seq, maxlen=30, padding="post", truncating="post")
val_ped = pad_sequences(val_seq, maxlen=30, padding="post", truncating="post")

In [140]:
train_ped = train_ped.astype(np.float32)
val_ped = val_ped.astype(np.float32) # переобразуем данные в тип float чтобы модель нас понял

In [111]:
len(train_ped[0])

50

In [141]:
rev = dict([(idx, word) for (word, idx) in wordss.items()])

def ddc(seq):
    return " ".join([rev.get(idx, "?") for idx in seq]) # даем индекс

In [151]:
from tensorflow.keras.regularizers import l2

model = keras.models.Sequential()
model.add(keras.layers.Embedding(len(counter), 32, input_length=30))
model.add(keras.layers.LSTM(64, dropout=0.5))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
model.add(keras.layers.Dense(11, activation="softmax"))

In [152]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 30, 32)            342368    
                                                                 
 lstm_9 (LSTM)               (None, 64)                24832     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 11)                363       
                                                                 
Total params: 369643 (1.41 MB)
Trainable params: 369643 (1.41 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [154]:
loss = keras.losses.SparseCategoricalCrossentropy()
optim = keras.optimizers.Adam(lr=0.001)
metrics = ["accuracy"]

model.compile(loss=loss, optimizer=optim, metrics=metrics)

In [136]:
y_train = y_train.astype(np.float32)
y_test  = y_test.astype(np.float32)

In [155]:
model.fit(train_ped, y_train, epochs=100, validation_data=(val_ped, y_test), verbose=2)

Epoch 1/100
265/265 - 109s - loss: 2.5191 - accuracy: 0.0894 - val_loss: 2.4074 - val_accuracy: 0.0881 - 109s/epoch - 411ms/step
Epoch 2/100
265/265 - 7s - loss: 2.3497 - accuracy: 0.1172 - val_loss: 2.2603 - val_accuracy: 0.1613 - 7s/epoch - 25ms/step
Epoch 3/100
265/265 - 8s - loss: 2.0943 - accuracy: 0.1802 - val_loss: 2.0569 - val_accuracy: 0.1913 - 8s/epoch - 28ms/step
Epoch 4/100
265/265 - 7s - loss: 1.9390 - accuracy: 0.2184 - val_loss: 2.0159 - val_accuracy: 0.2383 - 7s/epoch - 25ms/step
Epoch 5/100
265/265 - 8s - loss: 1.8261 - accuracy: 0.2723 - val_loss: 1.9706 - val_accuracy: 0.2510 - 8s/epoch - 31ms/step
Epoch 6/100
265/265 - 7s - loss: 1.7161 - accuracy: 0.3113 - val_loss: 2.0010 - val_accuracy: 0.2648 - 7s/epoch - 25ms/step
Epoch 7/100
265/265 - 8s - loss: 1.6094 - accuracy: 0.3475 - val_loss: 1.9581 - val_accuracy: 0.2858 - 8s/epoch - 29ms/step
Epoch 8/100
265/265 - 7s - loss: 1.5223 - accuracy: 0.3756 - val_loss: 1.9562 - val_accuracy: 0.3241 - 7s/epoch - 26ms/step
Epo

# ИТОГ В результате модель была обучен и показала точность 8-40%, когда я применил слой регуляризации и отсева, модель стала точной на 20-22%. Это был мой первый опыт разработки NLP в библиотеке Tensorflow